# 👁️ 에이전트의 Observability 활성화

> **개요**  
> OpenTelemetry를 활용하여 에이전트와의 상호 작용을 자동으로 기록하고 내보내는 방법을 알아봅니다.

---

## 📦 필요한 모듈 설치

In [ ]:
pip install opentelemetry-exporter-otlp-proto-grpc

---

## 🔧 OpenTelemetry 활성화 방법

관찰가능성을 활성화하는 가장 간단한 방법은 `configure_otel_providers`를 사용하는 것입니다.

In [ ]:
from agent_framework.observability import configure_otel_providers

# Reads OTEL_EXPORTER_OTLP_* environment variables automatically
configure_otel_providers()

---

## 🖥️ Console을 통한 Trace 확인

가장 기본적인 방법으로 Console에 trace가 출력되도록 설정하고 에이전트를 실행해 보겠습니다.

In [ ]:
from agent_framework import ChatAgent
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential
from agent_framework.observability import configure_otel_providers
from opentelemetry.sdk.trace.export import ConsoleSpanExporter
from opentelemetry.sdk._logs.export import ConsoleLogRecordExporter 

# Console에 trace와 log를 출력하도록 설정
configure_otel_providers(
    exporters=[
        ConsoleSpanExporter(),        # Trace를 콘솔에 출력
        ConsoleLogRecordExporter()    # Log를 콘솔에 출력
    ]
)

# Create the agent - telemetry is automatically enabled
agent = ChatAgent(
    chat_client=AzureOpenAIChatClient(credential=AzureCliCredential()),
    name="Joker",
    instructions="당신은 한국어로 말하는, 농담을 잘하는 어시스턴트입니다. 😄🎭"
)

# Run the agent
result = await agent.run("해적에 관한 농담을 해줘.")
print(result.text)

### 중요 ###
# 위의 코드를 Jyputer Notebook에서 실행하는 경우에는 Console에 Trace가 출력되지 않을 수 있습니다.
# 이 경우에는 Observability.py 파일을 별로도 만들어서 테스트 해야 합니다. 
# 이미 현재 폴더에는 완성된 Observability.py 파일이 있으므로, 해당 파일을 열어서 코드를 확인해 보고 실행합니다.
# 실행을 위해서는 우선 터미널을 열고, 가상환경을 활성화 한 후에 다음 명령어를 실행합니다.
#
# .\.venv\Scripts\Activate.ps1; 
# python .\MAF\Observability.py

---

### 📊 실행 결과

Console에서 `./MAF/Observability.py` 파일을 실행하면 다음과 같은 결과를 확인할 수 있습니다.

![console](images/maf-trace-console.png)

---

## 📈 Aspire Dashboard를 사용한 Trace 시각화

로컬에서 Trace를 확인하기 위해 뛰어난 텔레메트리 UI를 제공하는 **Aspire Dashboard**를 사용할 수 있습니다. Docker를 통해 로컬에서 실행됩니다.

### 🐳 Docker를 사용한 Aspire Dashboard 설치

> **⚠️ 사전 요구사항**  
> 로컬 머신에 Docker가 설치되어 있어야 합니다.  
> 설치가 필요한 경우 [Docker 설치 가이드](https://docs.docker.com/get-docker/)를 참고하세요.  
> Docker 설치가 어려운 경우 이번 실습은 건너뛰고 동료의 진행 상황을 참고하거나 코치의 데모를 확인하세요.

```bash
# Pull and run the Aspire Dashboard container
docker run --rm -it -d \
    -p 18888:18888 \
    -p 4317:18889 \
    --name aspire-dashboard \
    mcr.microsoft.com/dotnet/aspire-dashboard:latest
```

### 🌐 접속 정보

이 명령은 다음과 같이 대시보드를 시작합니다:

- **Web UI**: http://localhost:18888
- **OTLP endpoint**: http://localhost:4317 (애플리케이션이 텔레메트리 데이터를 전송)

### ⚙️ 환경변수 설정

`.env` 파일에 다음 환경변수를 추가합니다:

```bash
ENABLE_INSTRUMENTATION=true
OTEL_EXPORTER_OTLP_ENDPOINT=http://localhost:4317
```

환경변수를 추가한 후, 다시 로드합니다.

In [ ]:
from dotenv import load_dotenv
load_dotenv()  # .env 파일 로드

---

## ▶️ 에이전트 생성 및 실행

이제 에이전트를 실행하여 trace가 정상적으로 기록되는지 확인합니다.

In [ ]:
from agent_framework import ChatAgent
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential
from agent_framework.observability import configure_otel_providers
from opentelemetry.sdk.trace.export import ConsoleSpanExporter
from opentelemetry.sdk._logs.export import ConsoleLogRecordExporter 

# 환경변수에 지정된 OTEL_EXPORTER_OTLP_ENDPOINT로 전송됨 (Aspire Dashboard)
configure_otel_providers()

# Create the agent - telemetry is automatically enabled
agent = ChatAgent(
    chat_client=AzureOpenAIChatClient(credential=AzureCliCredential()),
    name="Joker",
    instructions="당신은 한국어로 말하는, 농담을 잘하는 어시스턴트입니다. 😄🎭"
)

# Run the agent
result = await agent.run("해적에 관한 농담을 해줘.")
print(result.text)

---

## 🔍 Aspire Dashboard에서 Trace 확인

### 📱 대시보드 접속

http://localhost:18888 로 접속하여 Aspire Dashboard가 실행 중인지 확인합니다.

### 🔐 로그인

로그인 토큰을 입력하라는 메시지가 나타나면, Docker Log에서 다음과 같은 링크를 확인할 수 있습니다.

> **💡 로그인 방법**  
> `Login to the dashboard at http://localhost:18888/login?t=********`  
> 해당 링크를 사용하거나, 로그인 화면에서 `login?t=` 이후의 토큰 값을 복사하여 입력합니다.

![image](images/maf-docker2.png)

### 📊 Trace 데이터 확인

Aspire Dashboard UI가 나타나면, **추적[Trace]** 탭을 클릭하여 에이전트와의 상호작용에서 생성된 Trace 데이터를 확인할 수 있습니다.

![Aspire Dashboard UI](images/maf-docker3.png)

---

## 📚 추가 자료

> **📖 자세한 내용**  
> Observability에 대한 더 자세한 정보는 공식 문서를 참고하세요.
>
> [Agent Framework Observability 문서](https://learn.microsoft.com/en-us/agent-framework/user-guide/observability?pivots=programming-language-python)